## 准备工作
* 获取access_token等参数信息

[通用参考](https://ai.baidu.com/ai-doc/REFERENCE/Ck3dwjhhu)

In [4]:
# A-1 获取access_token等参数信息
# encoding:utf-8
# client_id 为官网获取的AK， client_secret 为官网获取的SK
import requests 
import pandas as pd
import json

host = 'https://aip.baidubce.com/oauth/2.0/token?'
params = {
    "grant_type":"client_credentials", # grant_type=client_credentials&client_id
    "client_id":"sppC66DFagDjKGKkrz6Q6X91", # 【官网获取的AK】
    "client_secret":"sTCBeBNyVY5HEPoAa7bywbdxaPI7nPcp" # 【官网获取的SK】
}
response = requests.get(host,params=params)
display(pd.json_normalize(response.json()).T) # T改变为上下排列

access_token=response.json()["access_token"]
access_token

,0
refresh_token,25.175b8fd93ae55c5686d0dd1520523c78.315360000....
expires_in,2592000
session_key,9mzdCSEJW1T1NbCwgFspz9MfS5Hen+T6+CjNuHeoOQboIE...
access_token,24.27b83f91ed637b423509f845c4967626.2592000.16...
scope,public nlp_simnet nlp_wordemb nlp_comtag nlp_d...
session_secret,b1cff2aff49c01c389b2eb03efef2e6a


'24.27b83f91ed637b423509f845c4967626.2592000.1608522705.282335-23019635'

## 基础技术
### 词法分析
能够识别出文本串中的基本词汇（分词），对这些词汇进行重组、标注组合后词汇的词性，并进一步识别出命名实体
1. 分词
2. 词性标注
3. 专名识别

In [7]:
# B-1 词法分析API测试
def baidu_lexer(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer?access_token="+access_token
    data = {
        "text" : text  
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [10]:
content_lexer = baidu_lexer(text="不要温和地走进那良夜，老年应当在日暮时燃烧咆哮；怒斥，怒斥光明的消逝。虽然智慧的人临终时懂得黑暗有理，因为他们的话没有迸发出闪电，他们也并不温和地走进那个良夜。")

# 二进制转换GBK
content_lexer_str = str(content_lexer,encoding="GBK")
# str转换dict
content_lexer_dict =json.loads(content_lexer_str)
# 表格化清晰展示词法分析结果
pd.json_normalize(content_lexer_dict["items"])

,loc_details,byte_offset,uri,pos,ne,item,basic_words,byte_length,formal
0,[],0,,v,,不要,[不要],4,
1,[],4,,a,,温和,[温和],4,
2,[],8,,u,,地,[地],2,
3,[],10,,v,,走进,[走进],4,
4,[],14,,r,,那,[那],2,
5,[],16,,n,,良夜,"[良, 夜]",4,
6,[],20,,w,,，,[，],2,
7,[],22,,n,,老年,[老年],4,
8,[],26,,v,,应当,[应当],4,
9,[],30,,p,,在,[在],2,


### C 词向量表示

**通过词语的向量化来实现文本的可计算，帮助快速完成语义挖掘、相似度计算等应用**

* 语义召回
    * 对候选资源进行词向量表示，并构建向量表示基础上的快速索引召回技术，与传统的基于字词倒排索引方法不同，直接从语义相关性角度上给用户召回结果
    
* 个性化推荐
    * 基于用户的历史行为建模用户兴趣表示，学习用户与推荐候选之间的兴趣匹配度，实现对用户的个性化推荐

In [22]:
# C-1 词向量表示API测试
def baidu_word_emb_vec(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/word_emb_vec?access_token="+access_token
    data = {
        "word" : text   
    }
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data)

In [27]:
r_word_emb_vec = baidu_word_emb_vec("刘翔")
content_word_emb_vec = r_word_emb_vec.content
content_word_emb_vec_str = str(content_word_emb_vec,encoding="GBK")
content_word_emb_vec_str_dict =json.loads(content_word_emb_vec_str)
display(pd.json_normalize(content_word_emb_vec_str_dict).T)

,0
log_id,2025836341066374325
word,刘翔
vec,"[0.492945, -0.317705, -0.391092, 0.339131, 0.1..."


In [24]:
len(content_word_emb_vec_str_dict["vec"])

1024

### D 词义相似度

**通过词语向量化来计算两个词之间的相似度，满足高精度要求的业务场景需求**

* 应用场景：
    * 专名挖掘:
        * 通过词语间语义相关性计算寻找人名、地名、机构名等词的相关词，扩大专有名词的词典，更好的辅助应用
    * query改写:
        * 通过寻找搜索query中词语的相似词，进行合理的替换，从而达到改写query的目的，提高搜索结果的多样性

</br>

* 技术挖掘：
    * 词向量决定高精度
    * 词表收录
    * 深度学习训练

In [36]:
# D-1  词义相似度API测试
def baidu_word_emb_sim(word_1,word_2):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/word_emb_sim?access_token="+access_token
    data = {
        "word_1":word_1,
        "word_2":word_2
    }
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data )

In [37]:
r_word_emb_sim = baidu_word_emb_sim("马","驴") 
content_word_emb_sim = r_word_emb_sim.content
content_word_emb_sim_str = str(content_word_emb_sim ,encoding="GBK")
content_word_emb_sim_str_dict =json.loads(content_word_emb_sim_str)
pd.json_normalize(content_word_emb_sim_str_dict).T

,0
log_id,2886110170860016917
score,0.102383
words.word_2,驴
words.word_1,马


### E DNN语言模型
**判断一句话是否符合语言表达习惯，帮助您实现文本分析、纠错、对话等多种语义应用**

* 应用场景：
    * 拼写纠错
        * 基于句子上下文，计算纠错候选的语言模型概率。用于拼写纠错，提升用户体验
    * 对话系统
        * 判断用户输入的句子是否符合自然语言表达习惯，辅助对话系统进行决策
    * 机器翻译
        * 语言模型对翻译候选的打分作为最终译文的重要排序指标，提升翻译效果
        
</br>

* 思考：
    * 如何获取精准度？
        * 基于超大规模的网页数据进行训练，可增强模型具备较高的准确度
    * 可能用到的技术？
        * 基于词向量和深度学习技术，可解决传统方法中的数据稀疏问题和维度灾难问题，模型泛化能力强，效果优
    * （为什么有些小应用要免费给用户使用？获取用户信息和数据，以便于深度学习）

In [42]:
def baidu_dnnlm_cn(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/dnnlm_cn?access_token="+access_token
    data = {
        "text" : text   
    }
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,headers=headers,data=encode_data)

In [43]:
r_dnnlm_cn = baidu_dnnlm_cn("不要温和地走进那良夜，老年应当在日暮时燃烧咆哮；怒斥，怒斥光明的消逝。")

In [44]:
content_dnnlm_cn = r_dnnlm_cn.content
content_dnnlm_cn_str = str(content_dnnlm_cn,encoding="GBK")
content_dnnlm_cn_str_dict =json.loads(content_dnnlm_cn_str)
pd.json_normalize(content_dnnlm_cn_str_dict).T

,0
log_id,4026222010839543797
text,不要温和地走进那良夜，老年应当在日暮时燃烧咆哮；怒斥，怒斥光明的消逝。
items,"[{'word': '不', 'prob': 0.0019318}, {'word': '要..."
ppl,7342.39


In [45]:
content_dnnlm_cn_str_dict["items"]

[{'word': '不', 'prob': 0.0019318},
 {'word': '要', 'prob': 0.103819},
 {'word': '温和', 'prob': 5.40001e-05},
 {'word': '地', 'prob': 0.190028},
 {'word': '走进', 'prob': 0.00174401},
 {'word': '那良', 'prob': 2.11794e-07},
 {'word': '夜', 'prob': 2.32234e-05},
 {'word': '，', 'prob': 0.000421228},
 {'word': '老年', 'prob': 1.81822e-06},
 {'word': '应当', 'prob': 5.29011e-08},
 {'word': '在', 'prob': 0.00639291},
 {'word': '日暮', 'prob': 2.15439e-06},
 {'word': '时', 'prob': 0.015012},
 {'word': '燃烧', 'prob': 9.85762e-07},
 {'word': '咆哮', 'prob': 2.70406e-06},
 {'word': '；', 'prob': 0.000376566},
 {'word': '怒斥', 'prob': 3.40455e-07},
 {'word': '，', 'prob': 0.00546429},
 {'word': '怒斥', 'prob': 2.489e-05},
 {'word': '光明', 'prob': 0.000138307},
 {'word': '的', 'prob': 0.0423985},
 {'word': '消逝', 'prob': 1.21643e-05},
 {'word': '。', 'prob': 0.0248469}]